In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [2]:
iris = load_iris()
X = iris.data[:, 2:]  # petal length and width
y = iris.target

In [3]:
dtree_clsf = DecisionTreeClassifier(max_depth=3)
dtree_clsf.fit(X=X, y=y)

DecisionTreeClassifier(max_depth=3)

In [4]:
iris.feature_names[2:]

['petal length (cm)', 'petal width (cm)']

In [5]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
from sklearn.tree import export_graphviz

export_graphviz(
    decision_tree=dtree_clsf,
    out_file="iris_dtree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True,
)

! brew install graphviz

! dot -Tpng iris_dtree.dot -o iris_dtree.png

Ex. 1: Experimental data `make_moons()`

In [188]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4)

In [189]:
X.shape

(10000, 2)

In [190]:
y.shape

(10000,)

In [191]:
y = y.reshape(X.shape[0], 1)

In [192]:
y.shape

(10000, 1)

In [194]:
import numpy as np

dataset = np.hstack((X, y))

In [204]:
import pandas as pd

dataset = pd.DataFrame(dataset)

In [331]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [332]:
decision_tree_clf = DecisionTreeClassifier()

In [333]:
from sklearn.model_selection import GridSearchCV

parameters = [{"max_depth": [3, 4, 5, 6, 10, 12], "max_leaf_nodes": [8, 10, 12, 15, 20, 30]}]

grid_search = GridSearchCV(estimator=decision_tree_clf, param_grid=parameters, cv=5, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid=[{'max_depth': [3, 4, 5, 6, 10, 12],
                          'max_leaf_nodes': [8, 10, 12, 15, 20, 30]}],
             verbose=1)

In [334]:
grid_search.best_params_

{'max_depth': 10, 'max_leaf_nodes': 15}

In [335]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, grid_search.predict(X_train))

0.865875

Decision Trees Forest

In [337]:
from sklearn.model_selection import ShuffleSplit

samples = ShuffleSplit(n_splits=1000, train_size=80, test_size=20)
samples_sets = []

for mini_train_index, mini_test_index in samples.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    samples_sets.append((X_mini_train, y_mini_train))

In [343]:
from sklearn.base import clone

forest = [clone(grid_search.best_estimator_) for _ in range(1000)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, samples_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7895635

In [345]:
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [346]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [347]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8525